In [ ]:
# Compatibility with Python 3
from __future__ import print_function  # print('me') instead of print 'me'
from __future__ import division  # 1/2 == 0.5, not 0

In [ ]:
# Show figures inside the notebook
%matplotlib inline

In [ ]:
import numpy as np  # Python array library
import matplotlib.pyplot as plt  # Python plotting library

The major arteries in a T1 MRI image often have high signal (white when displaying in grayscale).

Our task is to see if we can pick out the courses of the [vertebral, basilar](http://en.wikipedia.org/wiki/Vertebral_artery#mediaviewer/File:Vertebral_artery_3D_AP.jpg) and [carotid](http://en.wikipedia.org/wiki/Internal_carotid_artery#mediaviewer/File:Magnetic_resonance_angiogram_of_segments_of_the_internal_carotid_artery.jpg) arteries on this image.

This time we are going to load an image using the `nibabel` library.  The image is `ds107_sub001_highres.nii` in the same directory as this notebook.

In [ ]:
import nibabel as nib

Try loading the image using nibabel, to make an image object. Use tab completion on `nib.` or look at the the [day0 introduction notebook](http://nbviewer.ipython.org/github/practical-neuroimaging/pna2015/blob/master/day0/what_is_an_image.ipynb) to work out how to do this.

In [ ]:
# img = ?
img = nib.load(filename='ds107_sub001_highres.nii')

Now get the image array data from the nibabel image object.  Don't forget to use tab completion on the image object if you can remember or don't know the methods of the object.

In [ ]:
# data = ?
data = img.dataobj

In [ ]:
data.shape

Try plotting a few slices over the third dimension to see whether you can see the arteries.  For example, to plot the first slice over the third dimension, you might use:

`plt.imshow(data[:, :, 0], cmap='gray')`


where `data` is your image array data.

In [ ]:
# Plotting some slices over the third dimension
fig, axes = plt.subplots(2,3, figsize=(12, 8))
axes = axes.flatten()
for i in range(6):
    axes[i].imshow(data[:,:,i*30], cmap='gray')

Now try looking for a good threshold so that you pick up only the very high signal in the brain.  A good place to start is to use ``plt.hist`` to get an idea of the spread of values within the volume and within the slice.

In [ ]:
# Zoom in on artery:
plt.imshow(data[75:125,100:150,60])

In [ ]:
# Here you might try plt.hist or something else to find a threshold
plt.hist(data[75:125,100:150,60].flatten(), bins=100)
plt.show()

In [ ]:
# Binarize using a threshold
artery_thresh = 325
plt.imshow(data[75:125,100:150,60] > artery_thresh)

Try making a binarized image with your threshold and displaying slices with that.  What structures are you picking up?

In [ ]:
# Maybe display some slices from the data binarized with a threshold
# Plotting some slices over the third dimension
fig, axes = plt.subplots(6,2, figsize=(8, 24))
center_volume = data[80:170,80:170, :]
for i in range(6):
    axes[i, 0].imshow(center_volume[:,:,i*30], cmap='gray')
    axes[i, 1].imshow(center_volume[:,:,i*30] > artery_thresh, cmap='gray')

Now try taking a 3D subvolume out of the middle of the image (the approximate middle in all three axes) to pick out a good subvolume of the image that still contains the big arteries.

In [ ]:
# Create a smaller 3D subvolume from the image data that still contains the arteries
center_volume_artery_slice_idx = [
    i for i in range(center_volume.shape[-1]) if np.sum(center_volume[:,:,i] > artery_thresh)
]
center_volume_artery_vol = center_volume[:, :, center_volume_artery_slice_idx]

In [ ]:
where_slice_break = np.where(np.diff(center_volume_artery_slice_idx) != 1)[0][0]

Try binarizing that with some thresholds to see whether you can pick out the arteries without much other stuff.  Hint - you might consider using `np.percentile` or `plt.hist` to find a good threshold.

In [ ]:
# Try a few plots of binarized slices and other stuff to find a good threshold
fig, axes = plt.subplots(6,2, figsize=(8, 24))
for ax_i, i in enumerate(range(0, where_slice_break, int(where_slice_break/6))):
    if ax_i < 6:
        axes[ax_i, 0].imshow(center_volume_artery_vol[:,:,i], cmap='gray')
        axes[ax_i, 1].imshow(center_volume_artery_vol[:,:,i] > artery_thresh, cmap='gray')

In [ ]:
center_volume_artery_vol_binary  = center_volume_artery_vol > artery_thresh

If you have a good threshold and a good binarized subset, see if you can see the arterial structure using the fancy function to plot the binarized image with a 3D rendering.

In [ ]:
# This function uses matplotlib 3D plotting and sckit-image for rendering
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure

def binarized_surface(binary_array):
    """ Do a 3D plot of the surfaces in a binarized image
    
    This uses scikit-image and some fancy commands that we don't
    need to worry about at the moment, to do the plot.
    """
    verts, faces = measure.marching_cubes_classic(binary_array, 0)
    fig = plt.figure(figsize=(10, 12))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], linewidths=0, alpha=0.3)
    mesh.set_edgecolor('k')
    ax.add_collection3d(mesh)
    ax.set_xlim(0, binary_array.shape[0])
    ax.set_ylim(0, binary_array.shape[1])
    ax.set_zlim(0, binary_array.shape[2])

In [ ]:
binarized_surface(binary_array=center_volume_artery_vol_binary)

In [ ]:
! pip install plotly

In [ ]:
# Use plotly to produce an interactive 3d plot
# NOTE: on VMware + Unbutu 18 on OSX, the resulting 3d plot can not be 'opened'. 
# openinbg the plot would lead to system crash.

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff

def binarized_surface(binary_array):
    """ Do a 3D plot of the surfaces in a binarized image
    
    This uses scikit-image and some fancy commands that we don't
    need to worry about at the moment, to do the plot.
    """
    verts, faces = measure.marching_cubes_classic(binary_array, 0)
    x,y,z = zip(*verts)
    colormap=['rgb(255,105,180)','rgb(255,255,51)','rgb(0,191,255)']
    fig = ff.create_trisurf(x=x,
                            y=y, 
                            z=z, 
                            plot_edges=False,
                            colormap=colormap,
                            simplices=faces,
                            title="Isosurface")
    plot(fig, filname='artery_3d.html')

For example, let's say you have a binarized subvolume of the original data called ``binarized_subvolume``.  You could do a 3D rendering of this binary image with:

`binarized_surface(binarized_subvolume)`

In [ ]:
# binarized_surface(binarized_subvolume)
binarized_surface(binary_array=center_volume_artery_vol_binary)